# COMP30760 ASSIGNMENT 2 - TASK 1 - 18439746 - Matthew O'Donnell
In this assignment we will collect stock price data from four HTML pages (Apple, Facebook, Intel, Microsoft).
This notebook covers **Task 1 - Data Collection and Preparation**

In [1]:
import requests
import pandas as pd
from pandas import read_html
import html5lib

In [2]:
stock_names = ["Apple", "Facebook", "Intel Corporation", "Microsoft Corporation"]
stock_codes = {"Apple":"AAPL", "Facebook":"FB","Intel Corporation":"INTC", "Microsoft Corporation":"MSFT"}
stock_urls = {"Apple":"http://mlg.ucd.ie/modules/COMP30760/stocks/data-aapl.html", 
              "Facebook":"http://mlg.ucd.ie/modules/COMP30760/stocks/data-fb.html",
              "Intel Corporation":"http://mlg.ucd.ie/modules/COMP30760/stocks/data-intc.html", 
              "Microsoft Corporation":"http://mlg.ucd.ie/modules/COMP30760/stocks/data-msft.html"}
df_companies = {}

# Data Collection and Preprocessing
## Parse Data 
We will define a function to first parse data from one html file and then a function to parse all of our files and store the resulting data frames in a dictionary

In [3]:
def parse_data(url):
    stocks = pd.io.html.read_html(url)
    df = pd.concat([stocks[0], stocks[1], stocks[2]], ignore_index=True) 
    df1 = pd.to_datetime(df[["Year", "Month", "Day"]])
    df1 = pd.DataFrame(df1, columns = ['Date'])
    date = df1["Date"]
    df = df.join(date)
    df = df.drop(columns = ['Year', 'Month', 'Day'])
    df.set_index('Date', inplace=True)
    df = df.round(decimals=2)
    return df

In [4]:
def parse_all_data():
    for stock_name in stock_names:
        df_companies.update({stock_name: parse_data(stock_urls[stock_name])})

We use our function to parse all our data and then we display some data from our Apple data frame

In [5]:
parse_all_data()
df_companies["Apple"].head(11)

,Stock,Open,High,Low,Close
Date,,,,,
2019-10-01,AAPL,56.27,57.06,56.05,56.15
2019-10-02,AAPL,55.76,55.90,54.48,54.74
2019-10-03,AAPL,54.61,55.24,53.78,55.21
2019-10-04,AAPL,56.41,56.87,55.97,56.75
2019-10-07,AAPL,56.57,57.48,56.46,56.76
2019-10-08,AAPL,56.46,57.01,56.08,56.10
2019-10-09,AAPL,56.76,56.95,56.41,56.76
2019-10-10,AAPL,56.98,57.61,56.83,57.52
2019-10-11,AAPL,58.24,59.41,58.08,59.05


We see above that one of our rows contains a date and a stock but no details about stocks (i.e. no Open, High, Low or Close figures). We also see that there are clearly some of the days of the month missing from our data set. First we will fill in the missing days of the month and fill in the stock name. Following this we will fill in the missing values of the stocks based on estimates.

## Adding missing dates and Replacing missing values
First we add the dates of each month that aren't included and also the relevant Stock Code

In [6]:
for stock_name in stock_names:
        df_companies[stock_name] = df_companies[stock_name].resample('D').max()
        df_companies[stock_name]["Stock"] = df_companies[stock_name]["Stock"].fillna(stock_codes[stock_name])

In [7]:
df_companies["Apple"].head(11)

,Stock,Open,High,Low,Close
Date,,,,,
2019-10-01,AAPL,56.27,57.06,56.05,56.15
2019-10-02,AAPL,55.76,55.90,54.48,54.74
2019-10-03,AAPL,54.61,55.24,53.78,55.21
2019-10-04,AAPL,56.41,56.87,55.97,56.75
2019-10-05,AAPL,NaN,NaN,NaN,NaN
2019-10-06,AAPL,NaN,NaN,NaN,NaN
2019-10-07,AAPL,56.57,57.48,56.46,56.76
2019-10-08,AAPL,56.46,57.01,56.08,56.10
2019-10-09,AAPL,56.76,56.95,56.41,56.76


Now we have to deal with the null values in our data frame. 

Let's check first to see how many null values are contained in our data frames. 

In [8]:
sum = 0
for stock_name in stock_names:
    sum+= df_companies[stock_name].isnull().values.sum()
sum

3942

### Replace Null Values
We create a new function to replace the missing values in our data frames with estimates based on nearby values to get an accurate representation of the data

In [9]:
def replace_missing_values(df):
    tmp = df 
    a = tmp.rolling(2).mean()
    b = tmp.iloc[::-1].rolling(2).mean()
    c = a.fillna(b).fillna(tmp).interpolate(method='nearest').ffill().bfill()
    tmp = tmp.fillna(c)
    tmp = pd.DataFrame(tmp, columns = ['Stock', 'Open', 'High', 'Low', 'Close'])
    return tmp

In [10]:
def replace_missing_all():
    for stock_name in stock_names:
        df_companies[stock_name] = replace_missing_values(df_companies[stock_name])

We then carry out this function and test to make sure that our function worked by checking to see that no more null values remain in our data frame. 

In [11]:
replace_missing_all()
sum = 0
for stock_name in stock_names:
    sum+= df_companies[stock_name].isnull().values.sum()
sum

0

In [12]:
df_companies["Apple"].head(11)

,Stock,Open,High,Low,Close
Date,,,,,
2019-10-01,AAPL,56.270,57.060,56.050,56.15
2019-10-02,AAPL,55.760,55.900,54.480,54.74
2019-10-03,AAPL,54.610,55.240,53.780,55.21
2019-10-04,AAPL,56.410,56.870,55.970,56.75
2019-10-05,AAPL,55.510,56.055,54.875,55.98
2019-10-06,AAPL,56.515,57.245,56.270,56.43
2019-10-07,AAPL,56.570,57.480,56.460,56.76
2019-10-08,AAPL,56.460,57.010,56.080,56.10
2019-10-09,AAPL,56.760,56.950,56.410,56.76


## Save Data Sets
Save the preprocessed versions of our four time series data sets

In [17]:
def save_files():
    for stock_name in stock_names:
        df_companies[stock_name].to_csv('/Users/matth/Documents/' + stock_name + 'Stocks.csv')

In [18]:
save_files()